In [32]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd
import re

In [33]:
def add_multicolumn(df, df2, new_col_name):
    tmp = df2.copy()    # make copy, otherwise df2 will be changed !!!
    tmp.columns = pd.MultiIndex.from_product([[new_col_name], df2.columns.tolist()])
    return pd.concat([df, tmp], axis=1)

def get_valid_filename(s):
    s = str(s).strip().replace(' ', '_')
    return re.sub(r'(?u)[^-\w.]', '', s)

In [34]:
model_path = '/users/Stathis/GoogleDrive/MLP/transfer_all_layers/'
nets =  list(filter(lambda x: os.path.isdir(os.path.join(model_path, x)), os.listdir(model_path)))
nets = [a for a in nets if 'resnet56' in a  or 'densenet121' in a]
nets.sort()

li = []
for net in nets:
    filename =  os.path.join(model_path, net) +'/result_outputs/test_summary.csv'
    df = pd.read_csv(filename, delimiter=",", index_col=None, header=0)
    nn = net.split('_')
    df['net'] = nn[3]
    df['source'] = nn[4]
    df['target'] = nn[5]
    df['step'] = nn[7]
    df['gamma'] = nn[9]
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [19]:
frame.set_index(['net', 'source', 'target'])

test_acc  test_loss step gamma
net         source target                                
densenet121 fgsm   fgsm      0.9145   0.358316   25   0.1
                   nat       0.9401   0.256466   25   0.1
                   nat       0.9388   0.254179   25   0.1
            nat    fgsm      0.9035   0.338362   25   0.1
                   nat       0.9340   0.271384   25   0.1
                   nat       0.9351   0.260235   25   0.1
                   pgd       0.8945   0.336705   25   0.1
            pgd    nat       0.9277   0.268576   25   0.1
                   nat       0.9261   0.296056   25   0.1
                   pgd       0.8938   0.357319   25   0.1
resnet56    fgsm   fgsm      0.8929   0.356749   25   0.1
                   fgsm      0.8906   0.370036   25   0.1
                   nat       0.9178   0.315059   25   0.1
                   nat       0.9189   0.355595   25   0.1
            nat    fgsm      0.9052   0.308398   25   0.1
                   fgsm      0.8925   0.355220   25   0.1
                   nat       0.9303   0.307776   20   0.4
                   nat       0.9250   0.324039   25   0.1
                   pgd       0.8810   0.439584   25   0.1
            pgd    nat       0.9184   0.314190   25   0.1
                   nat       0.9128   0.355118   25   0.1
                   pgd       0.8868   0.372155   25   0.1

In [47]:
model_path = '/users/Stathis/GoogleDrive/MLP/attack_results_pgd_7/'
res =  os.listdir(model_path)
res = [a for a in res if 'all_layers' in a]

In [53]:
import os
import json

In [108]:
model_path = '/users/Stathis/GoogleDrive/MLP/attack_results_pgd_7/'
nets =  list(os.listdir(model_path))
nets = [a for a in nets if 'all_layers' in a]
nets.sort()

attack_results = {}

for net in nets:
    fname = model_path+net
    with open(fname,'r+') as json_data:
        attack_results[net[24:-5]] = list(json.load(json_data).values())

In [125]:
df = pd.DataFrame.from_dict(attack_results, columns=['Clean', 
                                                     'FGSM_0.03125', 'PGD_0.03125',
                                                     'FGSM_0.0625', 'PGD_0.0625',
                                                     'FGSM_0.125', 'PGD_0.125',                                                    
                                                    ], orient='index')
# df=df.sort_values('Clean',ascending = False)
df=np.round(df*100,1)

In [126]:
for net in df.iterrows():
#     print(net[0])
    nn = net[0].split('_')
#     print(nn)
    df.loc[net[0],'net'] = nn[3]
    df.loc[net[0],'source'] = nn[4]
    df.loc[net[0],'target'] = nn[5]
    df.loc[net[0],'step'] = nn[7]
    df.loc[net[0],'gamma'] = nn[9]

In [135]:
df.set_index([ 'net','source', 'target', ])

Clean  FGSM_0.03125  PGD_0.03125  FGSM_0.0625  \
net         source target                                                  
densenet121 fgsm   fgsm     91.5          77.2         60.3         68.3   
                   nat      94.0          60.8         20.1         49.3   
                   nat      93.9          60.2         19.5         48.4   
            nat    fgsm     90.4          78.8         65.0         69.8   
                   nat      93.4          56.0         14.4         44.9   
                   nat      93.5          56.4         15.1         45.2   
                   pgd      89.4          75.4         60.5         66.4   
            pgd    nat      92.8          59.0         21.1         47.7   
                   nat      92.6          61.1         21.6         49.9   
                   pgd      89.4          77.6         60.3         67.2   
resnet56    fgsm   fgsm     89.3          72.5         53.7         59.5   
                   fgsm     89.1          72.8         56.2         60.1   
                   nat      91.8          60.2         22.6         48.3   
                   nat      91.9          59.9         22.0         49.2   
            nat    fgsm     90.5          78.2         69.7         68.0   
                   fgsm     89.3          75.3         62.5         64.6   
                   nat      93.0          58.1         16.3         46.1   
                   nat      92.5          58.8         17.3         47.2   
                   pgd      88.1          76.5         65.1         68.1   
            pgd    nat      91.8          60.8         20.0         50.8   
                   nat      91.3          62.5         22.1         52.0   
                   pgd      88.7          75.7         60.0         66.7   

                           PGD_0.0625  FGSM_0.125  PGD_0.125 step gamma  
net         source target                                                
densenet121 fgsm   fgsm          36.0        54.2       13.8   25   0.1  
                   nat            6.8        38.4        3.8   25   0.1  
                   nat            6.9        37.4        4.2   25   0.1  
            nat    fgsm          41.8        53.4       17.2   25   0.1  
                   nat            5.5        34.9        3.8   25   0.1  
                   nat            5.9        35.0        3.8   25   0.1  
                   pgd           35.8        52.8       14.2   25   0.1  
            pgd    nat            7.9        35.9        4.7   25   0.1  
                   nat            7.9        38.9        4.8   25   0.1  
                   pgd           35.7        52.1       13.9   25   0.1  
resnet56    fgsm   fgsm          28.4        44.0       11.8   25   0.1  
                   fgsm          30.7        45.1       13.0   25   0.1  
                   nat            9.1        37.2        5.5   25   0.1  
                   nat            8.5        39.1        5.2   25   0.1  
            nat    fgsm          48.6        50.2       21.4   25   0.1  
                   fgsm          39.6        48.6       17.2   25   0.1  
                   nat            6.6        36.2        4.5   20   0.4  
                   nat            6.7        36.2        4.6   25   0.1  
                   pgd           42.9        52.9       19.3   25   0.1  
            pgd    nat            8.8        40.2        5.4   25   0.1  
                   nat            9.6        41.0        6.0   25   0.1  
                   pgd           35.8        52.3       15.3   25   0.1